In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key = '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',
    model_name = "llama-3.1-70b-versatile"
    # other params...
)

response = llm.invoke("The first person to land on moon")
print(response)

content='The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 42, 'total_tokens': 117, 'completion_time': 0.3, 'prompt_time': 0.007185508, 'queue_time': 0.006447121, 'total_time': 0.307185508}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None} id='run-aa2a95bc-539b-4bcd-b0a3-72e59f52b935-0' usage_metadata={'input_tokens': 42, 'output_tokens': 75, 'total_tokens': 117}


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-42404")
page_data= loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware EngineerBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more than out

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job posting and return them in JSON format containing 
    following keys: `role`, `experience`,`skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data })
print(res.content)


```json
{
  "role": "Software Engineer",
  "experience": "2 years",
  "skills": [
    "Java",
    "NodeJS",
    "DynamoDB",
    "AWS",
    "Agile Practices using Scrum",
    "API Development",
    "CI/CD",
    "JSON and XML Markup Language",
    "Integration Testing",
    "Automation Testing",
    "Relational Databases such as Oracle"
  ],
  "description": "Develop, code, and configure test programs/systems while solutioning problems in order to meet defined digital product specifications and direction; Develop robust advanced analytics and machine learning solutions that have a direct impact on the business; Collaborate in the design and build of scalable software solutions; Implement and integrate new technologies while also managing and evolving existing platforms; Design and implement scalable applications that leverage prediction models and optimization programs to deliver data driven decisions that result in immense business impact; Assess a well-defined problem and lead the deve

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer',
 'experience': '2 years',
 'skills': ['Java',
  'NodeJS',
  'DynamoDB',
  'AWS',
  'Agile Practices using Scrum',
  'API Development',
  'CI/CD',
  'JSON and XML Markup Language',
  'Integration Testing',
  'Automation Testing',
  'Relational Databases such as Oracle'],
 'description': 'Develop, code, and configure test programs/systems while solutioning problems in order to meet defined digital product specifications and direction; Develop robust advanced analytics and machine learning solutions that have a direct impact on the business; Collaborate in the design and build of scalable software solutions; Implement and integrate new technologies while also managing and evolving existing platforms; Design and implement scalable applications that leverage prediction models and optimization programs to deliver data driven decisions that result in immense business impact; Assess a well-defined problem and lead the development of a technical solution that meets

In [11]:
import pandas as pd 
df = pd.read_csv("my_portfolio.csv")

In [12]:
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [16]:
import chromadb

import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

C:\Users\prg\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|███████| 79.3M/79.3M [00:02<00:00, 31.6MiB/s]


In [20]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{

In [19]:
job = json_res
job['skills']

['Java',
 'NodeJS',
 'DynamoDB',
 'AWS',
 'Agile Practices using Scrum',
 'API Development',
 'CI/CD',
 'JSON and XML Markup Language',
 'Integration Testing',
 'Automation Testing',
 'Relational Databases such as Oracle']

In [21]:

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Prasanth, a business development executive at Tesla. Tesla is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Tesla 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Tesla's portfolio: {link_list}
        Remember you are Prasanth, BDE at Tesla. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )


chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Your Business Needs

Dear Hiring Manager,

I came across your job posting for a Software Engineer with expertise in Java, NodeJS, DynamoDB, AWS, and Agile Practices using Scrum. I'd like to introduce you to Tesla, a leading AI & Software Consulting company that can help you find the perfect fit for this role.

At Tesla, we have a proven track record of delivering scalable software solutions that meet the needs of businesses like yours. Our team of experts has extensive experience in developing robust advanced analytics and machine learning solutions that drive business impact. We're confident that our capabilities align with your requirements, and we'd love to explore how we can support your organization.

Our portfolio showcases our expertise in various technologies, including:

* Java: https://example.com/java-portfolio
* NodeJS: https://example.com/react-portfolio (our React portfolio demonstrates our proficiency in NodeJS)
* AWS: h